In [ ]:
%%capture
import sys
if 'google.colab' in sys.modules:
    !pip install covasim optuna
    !git clone https://github.com/mazzio97/EpidemicModelLearning.git
    sys.path.append('EpidemicModelLearning')
    sys.path.append('EpidemicModelLearning/notebooks')
    sys.path.append('EpidemicModelLearning/notebooks/util')

# Calibration

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import covasim as cv
from util import data, calibration

sns.set_context('notebook')
sns.set_style('whitegrid')

pop_size = 450e3
pop_scale = 10
df = data.get_regional_data(scaling_factor=4.46e6/pop_size)

cols = set(df.columns) - {'date', 'new_tests'}
n_runs = 3
n_trials = 300

sample_weight = calibration.get_sample_weights(df, 'proportional')
custom_estimator = calibration.get_custom_estimator('mse', sample_weight)

In [ ]:
_, axes = plt.subplots(2, 2, figsize=(14, 8), sharex=True)

sns.lineplot(data=df, x='date', y='new_tests', ax=axes[0, 0]).set(title='New Tests', ylabel='')
sns.lineplot(data=df, x='date', y='cum_diagnoses', ax=axes[0, 1]).set(title='Cumulative Cases', ylabel='')
sns.lineplot(data=df, x='date', y='cum_deaths', ax=axes[1, 0]).set(title='Cumulative Deaths', ylabel='')
sns.lineplot(data=df, x='date', y='n_severe', label='n_severe', ax=axes[1, 1]).set(title='Active Cases')
sns.lineplot(data=df, x='date', y='n_critical', label='n_critical', ax=axes[1, 1]).set(ylabel='')

plt.tight_layout()

In [ ]:
default_params = dict(
    pop_type='hybrid',
    location='italy',
    start_day=df['date'].iloc[0],
    end_day=df['date'].iloc[-1],
    pop_size=pop_size / pop_scale,
    pop_scale=pop_scale,
    rescale=True,
    n_beds_hosp=pop_size * 370.4 / 100e3,
    n_beds_icu=pop_size * 14.46 / 100e3,
    quar_period=14,
    verbose=0
)

In [ ]:
constraint_orderings = [
    ('yellow_work_contacts', 'orange_work_contacts', 'red_work_contacts'),
    ('yellow_school_contacts', 'orange_school_contacts'),
    ('yellow_casual_contacts', 'orange_casual_contacts', 'red_casual_contacts'),
    ('init_imports', 'yellow_imports', 'orange_imports', 'red_imports'),
    ('init_beta', 'summer_beta'),
    ('winter_beta', 'summer_beta')
]

def objective(trial):
    # define learnable parameters
    initial_params = dict(
        pop_infected=trial.suggest_int('pop_infected', pop_size // 1e4, pop_size // 1e2, step=5),
        **default_params
    )
    intervention_params = dict(
        # CONTACT TRACING
        trace_prob=trial.suggest_float('trace_prob', 0.3, 0.9, step=1e-2),
        trace_time=trial.suggest_float('trace_time', 1.0, 7.0, step=1e-1),
        # SMART WORKING
        yellow_work_contacts=trial.suggest_float('yellow_work_contacts', 0.8, 1.0, step=1e-2),
        orange_work_contacts=trial.suggest_float('orange_work_contacts', 0.6, 1.0, step=1e-2),
        red_work_contacts=trial.suggest_float('red_work_contacts', 0.4, 0.9, step=1e-2),
        # SCHOOLS CLOSED
        yellow_school_contacts=trial.suggest_float('yellow_school_contacts', 0.4, 1.0, step=1e-2),
        orange_school_contacts=trial.suggest_float('orange_school_contacts', 0.4, 1.0, step=1e-2),
        # LOCKDOWN INTERVENTION
        yellow_casual_contacts=trial.suggest_float('yellow_casual_contacts', 0.6, 1.0, step=1e-2),
        orange_casual_contacts=trial.suggest_float('orange_casual_contacts', 0.2, 0.9, step=1e-2),
        red_casual_contacts=trial.suggest_float('red_casual_contacts', 0.0, 0.6, step=1e-2),
        # IMPORTED CASES
        init_imports=trial.suggest_float('init_imports', 5.0, 20.0, step=1e-1),
        yellow_imports=trial.suggest_float('yellow_imports', 4.0, 12.0, step=1e-1),
        orange_imports=trial.suggest_float('orange_imports', 2.0, 8.0, step=1e-1),
        red_imports=trial.suggest_float('red_imports', 0.0, 4.0, step=1e-1),
        # VIRAL LOAD REDUCTION
        init_beta=trial.suggest_float('init_beta', 0.01, 0.04, step=None),
        summer_beta=trial.suggest_float('summer_beta', 0.002, 0.02, step=None),
        winter_beta=trial.suggest_float('winter_beta', 0.01, 0.04, step=None),
        init_symp=trial.suggest_float('init_symp', 0.5, 5.0, log=True),
        init_sev=trial.suggest_float('init_sev', 0.5, 5.0, log=True),
        init_crit=trial.suggest_float('init_crit', 0.5, 5.0, log=True),
        init_death=trial.suggest_float('init_death', 0.5, 5.0, log=True)
    )
    # define and run simulations
    intervs = interventions.Interventions(df['date'].iloc[0], **intervention_params).get_all()
    sim = cv.Sim(pars=initial_params, interventions=intervs, datafile=df)
    msim = cv.MultiSim(sim)
    msim.run(n_runs=n_runs)
    # compute loss
    weights = {c: 1 for c in cols}
    mismatches = [s.compute_fit(keys=cols, weights=weights, estimator=custom_estimator).mismatch for s in msim.sims]
    violation = calibration.compute_violation(intervention_params, constraint_orderings)
    return sum(mismatches) / n_runs

study = op.create_study()
study.optimize(func=objective, n_trials=n_trials)

In [ ]:
results = pd.DataFrame([dict(objective=t.value, **t.params) for t in study.trials]).sort_values('objective')
summary = results.head(n_trials // 20).describe().loc[['count', 'min', 'max']]
summary = summary.append(pd.Series({'objective': study.best_value, **study.best_params}, name='best'))
summary = summary.transpose().astype({'count': 'int'})
summary

In [ ]:
for id, best_params in results.iloc[:3].iterrows():
  initial_params = {'pop_infected': best_params['pop_infected'], **default_params}
  interv_params = {k: v for k, v in best_params.items() if k != 'pop_infected'}
  intervs = interventions.Interventions(df['date'].iloc[0], **interv_params).get_all()
  sim = cv.Sim(pars=initial_params, interventions=intervs, datafile=df)
  msim = cv.MultiSim(sim)
  msim.run(n_runs=30)
  msim.mean()
  print(best_params)
  msim.plot(list(cols) + ['n_infectious', 'n_susceptible'])
  print('\n')